In [5]:
# Libraries
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

from model import BertForTokenClassification
import utils.NERutils as nu
import utils.QueryUtils as q

from transformers import AutoConfig, AutoTokenizer

from torch.utils.data import DataLoader, SubsetRandomSampler

# Define tokenizer
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Load data
train_path = "data/BIOtrain.parquet"
dev_path = "data/BIOdev.parquet"
test_path = "data/BIOtest.parquet"

FILTER = None

train_dataset = nu.NERdataset(dataset_path=train_path, tokenizer=bert_tokenizer, filter=FILTER)
dev_dataset = nu.NERdataset(dataset_path=dev_path, tokenizer=bert_tokenizer, filter=FILTER,tags=train_dataset.tags, index2tag=train_dataset.index2tag, tag2index=train_dataset.tag2index)
test_dataset = nu.NERdataset(dataset_path=test_path, tokenizer=bert_tokenizer, filter=FILTER,tags=train_dataset.tags, index2tag=train_dataset.index2tag, tag2index=train_dataset.tag2index)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
dev_loader = DataLoader(dev_dataset, batch_size=8, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

# Set device
if torch.backends.mps.is_available():
    device = torch.device("cpu")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


# Config
bert_model_name = "bert-base-multilingual-cased"
bert_config = AutoConfig.from_pretrained(
    bert_model_name, 
    num_labels=len(train_dataset.tags), 
    id2label=train_dataset.index2tag, 
    label2id=train_dataset.tag2index
)

# initialise model
model = BertForTokenClassification.from_pretrained(bert_model_name, config=bert_config, tags=train_dataset.tags, verbose=True).to(device)

# Load model
model.load_state_dict(torch.load("Trained_models/checkpoint_all_data.pt", map_location=device))

# Evaluate model
preds, targets = nu.evaluate_model(model=model, dataloader=test_loader, device=device)

# Convert ids to tags
preds = [*map(train_dataset.index2tag.get, list(preds))]
golds = [*map(train_dataset.index2tag.get, list(targets))]

f1score = nu.getF1ScoreFromLists(golds=golds, preds=preds)
print(f"{f1score = }")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 26%|██▌       | 48/183 [00:55<02:36,  1.16s/it]

In [2]:
len(dev_loader)

183